# Pandas

pandas의 심화 부분을 다뤄보도록 하겠습니다.

1. 데이터 불러오고
2. 데이터프레임으로 만들고
3. column을 생성 / 수정 / 삭제 하고
4. indexing과 slicing을 통해 원하는 조건에 맞는 데이터만 불러오고
5. groupby를 통해 조건에 따라 데이터를 그룹으로 나누어 살펴보기도 했습니다.

먼저 판다스에서 사용되는 자료형에 대해 간략하게 복습해보겠습니다.

---

![Imgur](https://i.imgur.com/rH0aXab.png)

### DataFrame
* 값들이 가로, 세로의 2차원 형태로 나열된 자료
* column: 데이터프레임의 가로 축
* row: 데이터프레임의 세로 축
* index: 각각의 row에 대한 이름
    * 인덱스를 활용하여 원하는 자료에 쉽게 접근할 수 있다.    

---

![Imgur](https://i.imgur.com/b3bxmhr.png)

### Series
* 값들이 1차원으로 쭉 나열된 자료형
* index: Series 안의 각각의 element에 붙여진 이름
    * 인덱스를 활용하여 원하는 자료에 쉽게 접근할 수 있다.    
* 데이터프레임에서 하나의 column을 불러오더라도, row를 불러오더라도 자료형은 series 형태

---

### 오늘 배울 내용 소개
    
* 멀티인덱싱
    * groupby - stack - unstack
    
    
* 데이터 값 변경하기
    * map, apply, applymap
    * string 데이터 다루기 
    
    
* 데이터프레임 합치기
    * merge, concat

### Movie Recommendation

* 수업에 사용할 데이터셋으로 영화 평점과 관련한 데이터셋입니다. <br>
아래 소스에서 약간의 전처리를 해주었습니다.
* source: [Kaggle - Movie Recommendation](https://www.kaggle.com/c/movie-recommendation) / [MovieLens](https://grouplens.org/datasets/movielens/)
* files
    * movies.csv : 영화 목록 데이터 (제목, 개봉연도, 장르)
    * users.csv : 유저 목록 데이터 (성별, 나이대, 직업, 거주지)
    * ratings.csv : 평점 목록 데이터 (유저, 영화, 평점)

먼저 각각 데이터 파일을 읽어들여 형태를 확인해보겠습니다!

In [1]:
import pandas as pd

movies = pd.read_csv("data/movies.csv",index_col="movie_id")
users = pd.read_csv("data/users.csv",index_col = "user_id")
ratings = pd.read_csv("data/ratings.csv", index_col = "id")

In [2]:
movies.head()

,title,genres
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


In [3]:
users.head()

,gender,age,occupation,zip
user_id,,,,
1,F,0,10,48067
2,M,50,16,70072
3,M,20,15,55117
4,M,40,7,02460
5,M,20,20,55455


In [4]:
ratings.head()

,user,movie,rating
id,,,
2783_1253,2783,1253,5
2783_589,2783,589,5
2783_1270,2783,1270,4
2783_1274,2783,1274,4
2783_741,2783,741,5


## 1. 멀티인덱싱

### Groupby

지난 시간 하나의 column을 기준으로 groupby 하는 방법을 배웠습니다. <br>
먼저 users dataframe에서 사용자의 나이대 / 성별 분포를 확인해봅시다. <br>

In [5]:
# age를 기준으로 groupby하여 각각 group에 해당하는 사용자 수 확인
users.groupby('age').size()

age
0      222
10    1103
20    2096
30    1193
40     550
50     876
dtype: int64

In [6]:
# gender를 기준으로 groupby하여 각각 group에 해당하는 사용자 수 확인
users.groupby('gender').size()

gender
F    1709
M    4331
dtype: int64

이때 나이와 성별 두가지 기준을 모두 활용하여 분류하려면 어떻게 해야할까요? <br>
이런 경우에도 groupby를 사용할 수 있습니다.<br>
적용하고 싶은 기준을 <font color='#d9544d'>리스트</font> 형태로 넣어주면 됩니다.

In [7]:
# age와 gender를 기준으로 groupby하여 저장하고 사용자 수 확인
user_age_gender = users.groupby(['age', 'gender']).size()
user_age_gender

age  gender
0    F           78
     M          144
10   F          298
     M          805
20   F          558
     M         1538
30   F          338
     M          855
40   F          189
     M          361
50   F          248
     M          628
dtype: int64

user_age_gender에서 index는 age와 gender의 [cartesian product](https://ko.wikipedia.org/wiki/%EA%B3%B1%EC%A7%91%ED%95%A9) 형태입니다! <br>
즉 age와 gender의 조합으로 이루어질 수 있는 모든 경우의 수를 포함합니다. <br>
이렇게 여러 기준이 섞인 index를 <font color = '#d9544d'>multi-index</font>라고 부릅니다. <br>

In [8]:
list(user_age_gender.index)

[(0, 'F'),
 (0, 'M'),
 (10, 'F'),
 (10, 'M'),
 (20, 'F'),
 (20, 'M'),
 (30, 'F'),
 (30, 'M'),
 (40, 'F'),
 (40, 'M'),
 (50, 'F'),
 (50, 'M')]

### Unstack
Series는 기본적으로 1차원의 자료형입니다. <br>
따라서 두 가지 기준을 Series로 나타내면 보기가 불편합니다.<br>
이때 unstack이라는 메소드를 활용하면 table(dataframe) 형태로 바꿀 수 있습니다.<br>
![Imgur](https://i.imgur.com/iiLmeHM.png)

In [9]:
# user_age_gender에 unstack() 적용
user_age_gender.unstack()

gender,F,M
age,,
0,78,144
10,298,805
20,558,1538
30,338,855
40,189,361
50,248,628


* multi-index (혹은 multi-level-index)가 적용된 series를 dataframe으로 변환
* defalut: index의 마지막 level을 column으로 설정

transpose()를 적용하면 축을 바꿀 수 있습니다. 

In [10]:
user_age_gender.unstack().transpose()

age,0,10,20,30,40,50
gender,,,,,,
F,78,298,558,338,189,248
M,144,805,1538,855,361,628


### Stack
Unstack과 반대로 dataframe을 (multi-indexed) Series로 바꾸는 메소드입니다. <br>
stack이란 기본적으로 쌓는다는 의미로 column을 현재 index 위에 쌓는 것입니다. <br>
![Imgur](https://i.imgur.com/TcQKg36.png)

In [11]:
user_age_gender.unstack().stack()

age  gender
0    F           78
     M          144
10   F          298
     M          805
20   F          558
     M         1538
30   F          338
     M          855
40   F          189
     M          361
50   F          248
     M          628
dtype: int64

---

## 2. 데이터 값 변경하기

이번에 살펴볼 주제는 dataframe 안에서 데이터를 일괄적으로 변경하는 방법입니다.<br>
먼저 아래 코드를 실행시켜볼게요!

In [14]:
myFrame = pd.DataFrame([["강예서","Science",15],
                        ["황우주","Design",14],
                        ["김혜나","Engineering",14]]
                       ,columns=["이름","팀","학번"])

In [15]:
myFrame

,이름,팀,학번
0,강예서,Science,15
1,황우주,Design,14
2,김혜나,Engineering,14


### Map

여기서 팀 칼럼명을 한글로 일괄적으로 바꾸고 싶습니다. 어떻게 하면 될까요? <br>
몇 가지 방법이 있지만, 두 경우로 나누어보겠습니다. <br>

1. 함수 사용
2. Dictionary 사용

#### 1.함수를 사용하는 경우

파이썬 시간에 함수를 배웠으니까 함수를 한 번 짜봅시다. <br>
아래 조건에 맞는 함수를 만들어보세요.<br>

* input: 팀명을 나타내는 영어 string 변수 x
* output:
    * input이 Science인 경우: 사이언스
    * input이 Design인 경우: 디자인
    * input이 Engineering인 경우: 엔지니어링

In [16]:
# 영어 팀명을 한글로 바꾸는 함수
def convertToKorean(x):
    if x == 'Science': return '사이언스'
    elif x == 'Design': return '디자인'
    else: return '엔지니어링'

이제 이 함수 **convertToKorean**를 활용해서 **myFrame**의 **팀** 컬럼을 바꾸고 싶습니다.<br>
이 때 **map**이라는 메소드를 활용합니다.

In [17]:
myFrame["팀"].map(convertToKorean)

0     사이언스
1      디자인
2    엔지니어링
Name: 팀, dtype: object

"팀" column의 영어 string이 한글 string으로 일괄 변경되었음을 확인할 수 있습니다.<br>
이 결과를 myFrame "팀" column에 넣어주려면 아래의 코드를 실행하면 됩니다.

In [18]:
myFrame["팀"] = myFrame["팀"].map(convertToKorean)
myFrame

,이름,팀,학번
0,강예서,사이언스,15
1,황우주,디자인,14
2,김혜나,엔지니어링,14


#### 2. Dictionary를 사용하는 경우
함수를 사용하지 않고, dictionary를 활용할 수도 있습니다.<br>
이번에는 위의 dataframe에서 팀 column을 다시 영어로 바꿔볼게요. <br>

* key: 한글 팀명 (무엇에서 바꾸고 싶은지)
* value: 영어 팀명 (무엇으로 바꾸고 싶은지)

In [19]:
myDict = {"사이언스":"Science", "디자인":"Design", "엔지니어링":"Engineering"}

위 dictionary는 한글 팀명을 영어 팀명으로 **매핑**하는 dictionary입니다.<br>
마찬가지로 **map** 메소드를 활용하면 column을 일괄적으로 변경할 수 있습니다.

In [20]:
myFrame["팀"].map(myDict)

0        Science
1         Design
2    Engineering
Name: 팀, dtype: object

myFrame에 변경을 적용하고 싶다면, 마찬가지로 아래와 같이 코드를 짜면 됩니다.

In [21]:
myFrame["팀"] = myFrame["팀"].map(myDict)
myFrame

,이름,팀,학번
0,강예서,Science,15
1,황우주,Design,14
2,김혜나,Engineering,14


### Apply

dataframe의 column안에 있는 값을 규칙에 따라 일괄적으로 변경하는 데에 <br>
map이라는 메소드를 활용해보았습니다. <br>
이와 비슷한 메소드로 apply가 있습니다.<br>

In [22]:
myFrame["팀"] = myFrame["팀"].apply(convertToKorean)
myFrame

,이름,팀,학번
0,강예서,사이언스,15
1,황우주,디자인,14
2,김혜나,엔지니어링,14


<font color = "#FF0000"> 그런데! 아래의 코드를 실행하면 에러가 납니다! </font>

In [ ]:
myFrame["팀"] = myFrame["팀"].apply(myDict)
myFrame

그 이유는 map과 apply는 조금 다르기 때문입니다. 
map과 apply의 차이는 아래와 같습니다.
* map: 매핑관계를 이용하여 컬럼의 값을 일괄적으로 변경한다.
    * 매핑관계 - function, dictionary, series
* apply: 컬럼에 일정한 <font color = "#d9544d">함수</font>를 적용(apply)해준다.

위의 예에서 apply에 dictionary를 이용한 매핑을 시도했기 때문에 에러가 뜬거죠!

### Applymap
이번에 살펴볼 내용은 daframe 안에 있는 **모든** 값을 바꿔주는 방법입니다.<br>
먼저 아래 코드를 실행해주세요.

In [23]:
data = [["2012000000",100,90,100,90],
        ["2015000000",80,100,92,10],
        ["2016000000",50,60,40,50],
        ["2017999999",100,10,0,100]]

assignment = pd.DataFrame(data,columns=["student_id",
                                        "1st","2nd","3rd","4th"]).set_index("student_id")

In [24]:
assignment

,1st,2nd,3rd,4th
student_id,,,,
2012000000,100,90,100,90
2015000000,80,100,92,10
2016000000,50,60,40,50
2017999999,100,10,0,100


assignment는 학생들의 과제 점수를 나타내는 dataframe입니다. <br>
각각의 과제가 100점 만점이라고 가정해봅시다. <br>
그런데 실제 점수를 계산할 때 각각의 점수가 10점 만점으로 반영된다고 합니다.<br>
그래서 assignment의 모든 값을 10점 만점으로 환산하고 싶은데, 어떻게 하면 될까요?<br><br>
이때 사용하는 메소드가 **applymap**입니다.<br>
applymap()안에는 apply와 마찬가지로 <font color="#d9544d">함수만 들어갈 수 있습니다.</font> <br>
lambda 식을 활용해서 위 dataframe의 모든 값을 10점 만점으로 환산해보세요.

In [25]:
# 각 과제점수를 10점 만점으로 환산
assignment = assignment.applymap(lambda x:x/10)

In [26]:
assignment

,1st,2nd,3rd,4th
student_id,,,,
2012000000,10.0,9.0,10.0,9.0
2015000000,8.0,10.0,9.2,1.0
2016000000,5.0,6.0,4.0,5.0
2017999999,10.0,1.0,0.0,10.0


이제 10점 만점으로 환산했을 경우, 학생별 과제 총점을 매겨보려고 합니다.<br>
어떻게 할 수 있을까요?

#### 1. 모든 column을 더한다.

In [27]:
assignment["1st"] + assignment["2nd"] + assignment["3rd"] + assignment["4th"]

student_id
2012000000    38.0
2015000000    28.2
2016000000    20.0
2017999999    21.0
dtype: float64

#### 2. apply 메소드를 사용한다.
* axis = 0: 세로 방향 (row들의 합계)
* axis = 1: 가로 방향 (column들의 합계)

In [28]:
assignment.apply(sum, axis = 1)

student_id
2012000000    38.0
2015000000    28.2
2016000000    20.0
2017999999    21.0
dtype: float64

apply는 "dataframe"의 메소드이면서 "series"의 메소드입니다. <br>
그래서 아까 설명한 apply랑은 조금 다른 방식으로 작동하는 것을 볼 수 있습니다.<br><br>
axis가 0이면 <font color="#d9544d">한 column을 series로 생각</font>하여 apply한 것과 같고, <br>
axis가 1이면 <font color="#d9544d">한 row를 series로 생각</font>하여 apply한 것과 같습니다. <br>

#### 3. sum() 메소드를 사용한다.

In [29]:
assignment.sum(axis = 1)

student_id
2012000000    38.0
2015000000    28.2
2016000000    20.0
2017999999    21.0
dtype: float64

어떤 방법이 됐든 개인별 과제 점수 총합을 새로운 컬럼으로 만들고 싶다면 아래와 같이 코드를 짤 수 있습니다.

In [30]:
assignment["total"] = assignment.sum(axis = 1)

In [31]:
assignment

,1st,2nd,3rd,4th,total
student_id,,,,,
2012000000,10.0,9.0,10.0,9.0,38.0
2015000000,8.0,10.0,9.2,1.0,28.2
2016000000,5.0,6.0,4.0,5.0,20.0
2017999999,10.0,1.0,0.0,10.0,21.0


이러한 dataframe, series의 일괄 변경은 데이터 전처리에 있어서 강력한 도구가 됩니다.<br>
반복문을 써서 데이터를 변경시켜줄 필요가 없기 때문이죠.<br>
**절대** dataframe 안에서 **<font color="#d9544d">반복문(for / while 등..)을 쓰지 마세요.</font>** <br>
속도가 매우 느립니다. <br>
대신 지금 배운 map, apply, applymap을 사용하세요. <br><br>


즉 정리하자면 
* map은 <font color = "#4286f4">series</font>의 각 element에 매핑을 일괄적용하는 메소드 (함수 포함)
* applymap은 <font color = "#edbc40">dataframe</font>의 각 element에 함수를 일괄적용하는 메소드
* apply는 <font color = "#4286f4">series</font>/<font color = "#edbc40">dataframe</font>의 각 element(series)/series(dataframe)에 함수를 일괄적용하는 메소드

### String 데이터 다루기
string으로 구성된 column 혹은 index 뒤에 **.str**를 붙이면 <br>
string에 적용할 수 있는 함수 / 연산들을 똑같이 쓸 수 있습니다.<br>
assignment 데이터프레임의 index에서, "입학년도" 정보를 뽑아볼게요.

In [35]:
# 학번의 상위 4자리를 입학년도로 사용
assignment["admission_year"] = assignment.index.str[:4]

In [36]:
assignment

,1st,2nd,3rd,4th,total,admission_year
student_id,,,,,,
2012000000,10.0,9.0,10.0,9.0,38.0,2012
2015000000,8.0,10.0,9.2,1.0,28.2,2015
2016000000,5.0,6.0,4.0,5.0,20.0,2016
2017999999,10.0,1.0,0.0,10.0,21.0,2017


슬라이싱 뿐만 아니라 split(), strip()등의 다양한 string 메소드를 사용할 수 있습니다.
* split()
* strip()

## 3. 데이터프레임 합치기
이번 시간에는 서로 다른 두 dataframe을 합치는 방법에 대해 알아보겠습니다. 

### Merge
먼저 아래의 코드를 실행해봅시다. 

In [41]:
scores = pd.DataFrame([["예서",100],["예빈",80],["혜나",99],
                       ["세리",70],["세리",85]],columns=["name","score"])
years = pd.DataFrame([["예서",1996],["예빈",2006],
                      ["세리",1997],["우주",2000]],columns=["name","year"])

In [42]:
# 멤버별 점수가 포함된 dataframe
scores

,name,score
0,예서,100
1,예빈,80
2,혜나,99
3,세리,70
4,세리,85


In [43]:
# 멤버별 출생년도를 나타내는 dataframe
years

,name,year
0,예서,1996
1,예빈,2006
2,세리,1997
3,우주,2000


이때 score와 years 2개의 dataframe을 어떻게 합칠 수 있을까요? <br>
즉 멤버별 이름, 점수, 출생년도를 모두 포함한 dataframe을 어떻게 만들 수 있을까요? <br>
이때 사용되는 것이 **merge**라는 메소드입니다. 

![join](https://letsdobigdata.files.wordpress.com/2016/03/joins.png)

위 그림은 여러가지 merge의 방법에 대해 나타낸 그림입니다. <br>
하나의 column을 기준으로 두 dataframe을 하나로 합칠 때
<font color = '#d9544d'>한쪽에만 존재하는 데이터를 어떻게 처리할 것인가</font>에 따라 그 결과가 달라집니다. <br>
<br>
예를 들어 scores와 years를 'name' 칼럼을 기준으로 합친다고 할 때, <br>
scores나 years 중 한 데이터셋에만 존재하는 사람(지효, 쯔위)을 처리하는 방법에 따라 <br>
merge의 종류가 정해집니다.<br>
<br>
아래는 그 예시입니다. <br>
how는 merge의 방법, on은 기준이 되는 칼럼명을 나타냅니다. <br>

In [44]:
# inner join - 두 데이터프레임에 모두 존재하는 데이터만
scores.merge(years, how='inner', on='name')

,name,score,year
0,예서,100,1996
1,예빈,80,2006
2,세리,70,1997
3,세리,85,1997


In [45]:
# left join - scores에 존재하는 데이터 기준
scores.merge(years, how='left', on='name')

,name,score,year
0,예서,100,1996.0
1,예빈,80,2006.0
2,혜나,99,NaN
3,세리,70,1997.0
4,세리,85,1997.0


In [46]:
# right join - years에 존재하는 데이터 기준
scores.merge(years, how='right', on='name')

,name,score,year
0,예서,100.0,1996
1,예빈,80.0,2006
2,세리,70.0,1997
3,세리,85.0,1997
4,우주,NaN,2000


In [47]:
# full join - 두 데이터프레임 중 어느 한 곳에 존재하는 
scores.merge(years, how='outer', on='name')

,name,score,year
0,예서,100.0,1996.0
1,예빈,80.0,2006.0
2,혜나,99.0,NaN
3,세리,70.0,1997.0
4,세리,85.0,1997.0
5,우주,NaN,2000.0


#### Q. 두 dataframe의 key column명이 다른 경우는 어떻게 할까요? 
(key column은 기준이 되는 column을 말합니다.) <br>
예를 들어 years의 name 칼럼명을 바꿔 years1 로 저장해보겠습니다. <br>
이때 'on' 대신에 'left_on'과 'right_on'을 사용하면 됩니다.

In [48]:
years1 = years.rename(columns={'name':'이름'})
scores.merge(years1, left_on="name", right_on="이름", how="inner")

,name,score,이름,year
0,예서,100,예서,1996
1,예빈,80,예빈,2006
2,세리,70,세리,1997
3,세리,85,세리,1997


#### Q. 기준이 되는 key column이 index에 있는 경우에는 어떻게 할까요? 
예를 들어 years의 name 칼럼을 index로 바꿔 years2로 저장해보겠습니다. <br>
이때 right/right_index=True를 사용하면 됩니다.

In [49]:
# years 데이터프레임의 key가 index에 있을 때
years2 = years.set_index('name')
scores.merge(years2, left_on="name", right_index=True, how="inner")

,name,score,year
0,예서,100,1996
1,예빈,80,2006
3,세리,70,1997
4,세리,85,1997


In [50]:
# scores 데이터프레임의 key가 index에 있을 때
scores2 = scores.set_index('name')
scores2.merge(years, left_index=True, right_on="name", how="inner")

,score,name,year
0,100,예서,1996
1,80,예빈,2006
2,70,세리,1997
2,85,세리,1997


In [51]:
# 두가지 데이터프레임의 key가 모두 index에 있을 때
out = scores2.merge(years2, left_index=True, right_index=True, how="inner")
out

,score,year
name,,
세리,70,1997
세리,85,1997
예빈,80,2006
예서,100,1996


만약 위 데이터프레임에서 key를 column으로 만들고 싶다면, <br>
merge 이후 reset_index()를 해주면 됩니다! <br>

In [52]:
out.reset_index()

,name,score,year
0,세리,70,1997
1,세리,85,1997
2,예빈,80,2006
3,예서,100,1996


## Concat
concatenate의 줄임말로 서로 다른 두개의 dataframe을 아래로 이어붙일 때 사용합니다. <br>
ratings.csv 파일과 test.csv 파일을 활용해봅시다. <br>

In [63]:
ratings = pd.read_csv("data/ratings.csv", index_col="id")
test = pd.read_csv("data/test.csv")

In [64]:
ratings.head()

,user,movie,rating
id,,,
2783_1253,2783,1253,5
2783_589,2783,589,5
2783_1270,2783,1270,4
2783_1274,2783,1274,4
2783_741,2783,741,5


In [65]:
test.head()

,user,rating,movie
0,1,3.5,1193
1,1,3.5,661
2,1,3.5,914
3,1,3.5,3408
4,1,3.5,2355


먼저 test 데이터프레임도 ratings와 같은 형식으로 바꿔줄게요. <br>
id는 **'user_movie'**의 형태를 가지고 있습니다. <br>
따라서 test 데이터프레임에서도 user, movie 칼럼을 이용해 id 칼럼을 만들고 <br>
이를 index로 바꿔보겠습니다. 

In [66]:
# id 칼럼 만들기
test['id'] = test['user'].map(str)+"_"+test['movie'].map(str)

In [67]:
# id 칼럼 index로 만들기
test = test.set_index('id')

In [68]:
test.head()

,user,rating,movie
id,,,
1_1193,1,3.5,1193
1_661,1,3.5,661
1_914,1,3.5,914
1_3408,1,3.5,3408
1_2355,1,3.5,2355


이제 이 두개의 dataframe을 concat해보겠습니다. 

In [69]:
# ratings dataframe 뒤에 test dataframe을 concatenate
output = pd.concat([ratings, test])
output.head()

/Users/beatricelee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,movie,rating,user
id,,,
2783_1253,1253,5.0,2783
2783_589,589,5.0,2783
2783_1270,1270,4.0,2783
2783_1274,1274,4.0,2783
2783_741,741,5.0,2783


ratings, test, output의 shape을 비교해보면 바르게 concat되었음을 확인할 수 있습니다!

In [70]:
print(ratings.shape, test.shape, output.shape)

(500100, 3) (500109, 3) (1000209, 3)
